In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
import pickle
import torch
from torch import nn
import yaml
from pathlib import Path
from collections import OrderedDict

import proxyClassifier
import perturbation_model
from proxyClassifier import proxy_clf_network, proxy_clf

from perturbation_model import perturb_network, perturb_clf

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

try:
    file_path = os.path.realpath(__file__)
    base_path =  os.path.dirname(basepath)
except:
    base_path = './'
CONFIG_FILE = os.path.join(base_path, 'config.yaml')
ID_COL = 'PanjivaRecordID'


# Globals
DIR = None
Data_Dir = None
Anom_Dir = None
data_count = None 
clf_batch_size = None
clf_model_save_dir = None 
perturbModel_train_epochs = None
def setup_config(_dir):
    global DIR, Data_Dir, Anom_Dir
    global CONFIG_FILE
    global data_count
    global clf_batch_size
    global clf_model_save_dir
    global base_path
    global perturbModel_train_epochs
    
    with open(CONFIG_FILE,'r') as fh:
        CONFIG = yaml.safe_load(fh)
    Data_Dir = CONFIG['data_loc']
    Anom_Dir = CONFIG['anom_data_path']
    DIR = _dir
    data_count = CONFIG['data_count']
    clf_batch_size = CONFIG['clf_batch_size']
    clf_model_save_dir = os.path.join(base_path, './clf_model_save_dir', DIR)
    Path(clf_model_save_dir).mkdir(exist_ok=True, parents=True)
    Path(clf_model_save_dir).mkdir(exist_ok=True, parents=True)
    perturbModel_train_epochs = CONFIG['perturbModel_train_epochs']
    return

setup_config('us_import1')

In [2]:
def get_domain_dims():
    global DIR, Data_Dir
    with open(os.path.join(Data_Dir, DIR, 'domain_dims.pkl'), 'rb') as fh:
        domain_dims = OrderedDict(pickle.load(fh))
    return domain_dims

In [3]:
def proxy_training_data():
    global DIR, Data_Dir, Anom_Dir, data_points, ID_COL, data_count
    # normal_instances
    path_normal = os.path.join(Data_Dir, DIR, 'train_data.csv')
    df_normal = pd.read_csv(path_normal,index_col=None)
    
    path_anom = os.path.join(Anom_Dir, DIR, 'anomalies_1_1.csv')
    df_anomalies_1 = pd.read_csv(path_normal,index_col=None)
    
    path_anom = os.path.join(Anom_Dir, DIR, 'anomalies_1_2.csv')
    df_anomalies_2 = pd.read_csv(path_normal,index_col=None)
    
    df_anomalies = df_anomalies_1.append(df_anomalies_2,ignore_index=True)
    
    
    if data_count > 0:
        if len(df_anomalies) < data_count:
            df_anomalies = df_anomalies.sample(n = data_count, replace=True)
        else:
            df_anomalies = df_anomalies.sample(n = data_count)

        df_normal = df_normal.sample(n = data_count) 
        
    # create X, Y
    Y1 = np.ones([len(df_normal)])
    Y2 = np.zeros([len(df_anomalies)])
    
    try: del df_normal[ID_COL]
    except: pass
    try: del df_anomalies[ID_COL]
    except: pass
    X1 = df_normal.values
    X2 = df_anomalies.values
    
    X = np.vstack([X1,X2])
    Y = np.hstack([Y1,Y2])            
    
    return X,Y
    

In [4]:
def get_proxy_clf():
    """
    Train model  and store
    """
    global clf_batch_size, DIR
    global DEVICE
    global clf_model_save_dir
    domain_dims = get_domain_dims()
    
    network = proxy_clf_network(
        list(domain_dims.values())
    )
    clf_obj = proxy_clf(
            model = network,
            dataset = DIR, 
            save_dir = clf_model_save_dir,
            batch_size= clf_batch_size,
            device = DEVICE    
        )
    
    try:
        clf_obj.load_model()
    except:
        
        train_X, train_Y = proxy_training_data()
        clf_obj.fit(train_X, train_Y.reshape([-1,1]), num_epochs=200)
        clf_obj.save_model()
    
    return clf_obj

In [5]:
clf_obj = get_proxy_clf()

Device cuda:0


In [6]:
def train_pert_model():
    global DEVICE, DIR
    domain_dims = get_domain_dims()
    clf_obj = get_proxy_clf()
    clf_network = clf_obj.model
    
    perturb_network_obj  = perturb_network(
        list(domain_dims.values())
    )
    
    perturb_clf_obj = perturb_clf(
        perturb_model = perturb_network_obj,
        clf_model = clf_network,
        dataset = DIR,
        device = DEVICE
    )
    # Train the model 
    
    return perturb_clf_obj

In [7]:
perturb_clf_obj = train_pert_model()

Device cuda:0


In [8]:
train_X, train_Y = proxy_training_data()

train_X.shape

train_Y.shape

(20000,)

In [9]:
loss_values = perturb_clf_obj.fit(X = train_X[-1000:], Y = np.abs(1 - train_Y[-1000:]).reshape([-1,1]), num_epochs = perturbModel_train_epochs)

  1%|          | 10/1000 [00:00<00:36, 26.96it/s]

[Epoch] 0  | batch 0 | Loss 1.748657
[Epoch] 1  | batch 0 | Loss 1.781845
[Epoch] 2  | batch 0 | Loss 1.983910
[Epoch] 3  | batch 0 | Loss 1.955962
[Epoch] 4  | batch 0 | Loss 1.920985
[Epoch] 5  | batch 0 | Loss 2.030923
[Epoch] 6  | batch 0 | Loss 2.083749
[Epoch] 7  | batch 0 | Loss 2.339351
[Epoch] 8  | batch 0 | Loss 2.233174
[Epoch] 9  | batch 0 | Loss 2.075572
[Epoch] 10  | batch 0 | Loss 1.855489
[Epoch] 11  | batch 0 | Loss 2.308291
[Epoch] 12  | batch 0 | Loss 2.140818
[Epoch] 13  | batch 0 | Loss 1.858302
[Epoch] 14  | batch 0 | Loss 2.053906
[Epoch] 15  | batch 0 | Loss 2.384487
[Epoch] 16  | batch 0 | Loss 2.008908


  3%|▎         | 26/1000 [00:00<00:18, 52.49it/s]

[Epoch] 17  | batch 0 | Loss 1.966647
[Epoch] 18  | batch 0 | Loss 2.172150
[Epoch] 19  | batch 0 | Loss 2.151794
[Epoch] 20  | batch 0 | Loss 2.093338
[Epoch] 21  | batch 0 | Loss 1.764663
[Epoch] 22  | batch 0 | Loss 1.881285
[Epoch] 23  | batch 0 | Loss 1.833573
[Epoch] 24  | batch 0 | Loss 2.060652
[Epoch] 25  | batch 0 | Loss 2.118948
[Epoch] 26  | batch 0 | Loss 1.873251
[Epoch] 27  | batch 0 | Loss 1.937286
[Epoch] 28  | batch 0 | Loss 1.927108
[Epoch] 29  | batch 0 | Loss 2.081978
[Epoch] 30  | batch 0 | Loss 1.942453
[Epoch] 31  | batch 0 | Loss 1.958050
[Epoch] 32  | batch 0 | Loss 1.630450
[Epoch] 33  | batch 0 | Loss 1.924700


  4%|▍         | 44/1000 [00:00<00:14, 67.87it/s]

...

[Epoch] 952  | batch 0 | Loss 1.828860
[Epoch] 953  | batch 0 | Loss 2.024080
[Epoch] 954  | batch 0 | Loss 1.723251
[Epoch] 955  | batch 0 | Loss 1.944839
[Epoch] 956  | batch 0 | Loss 1.974316
[Epoch] 957  | batch 0 | Loss 2.199961
[Epoch] 958  | batch 0 | Loss 1.718175
[Epoch] 959  | batch 0 | Loss 1.775737
[Epoch] 960  | batch 0 | Loss 1.982655
[Epoch] 961  | batch 0 | Loss 2.073966
[Epoch] 962  | batch 0 | Loss 2.032851
[Epoch] 963  | batch 0 | Loss 1.784951
[Epoch] 964  | batch 0 | Loss 1.760723
[Epoch] 965  | batch 0 | Loss 2.356742
[Epoch] 966  | batch 0 | Loss 1.839314
[Epoch] 967  | batch 0 | Loss 1.957399
[Epoch] 968  | batch 0 | Loss 1.747418


 98%|█████████▊| 979/1000 [00:12<00:00, 82.61it/s]

[Epoch] 969  | batch 0 | Loss 1.843012
[Epoch] 970  | batch 0 | Loss 1.915320
[Epoch] 971  | batch 0 | Loss 2.357218
[Epoch] 972  | batch 0 | Loss 2.026498
[Epoch] 973  | batch 0 | Loss 1.895410
[Epoch] 974  | batch 0 | Loss 2.021171
[Epoch] 975  | batch 0 | Loss 1.839972
[Epoch] 976  | batch 0 | Loss 1.886445
[Epoch] 977  | batch 0 | Loss 1.843869
[Epoch] 978  | batch 0 | Loss 2.313621
[Epoch] 979  | batch 0 | Loss 1.896933
[Epoch] 980  | batch 0 | Loss 1.876756
[Epoch] 981  | batch 0 | Loss 1.911339
[Epoch] 982  | batch 0 | Loss 2.164487
[Epoch] 983  | batch 0 | Loss 1.815938
[Epoch] 984  | batch 0 | Loss 2.153142
[Epoch] 985  | batch 0 | Loss 1.819600


100%|██████████| 1000/1000 [00:12<00:00, 80.49it/s]

[Epoch] 986  | batch 0 | Loss 1.637998
[Epoch] 987  | batch 0 | Loss 1.958771
[Epoch] 988  | batch 0 | Loss 1.965782
[Epoch] 989  | batch 0 | Loss 2.002327
[Epoch] 990  | batch 0 | Loss 2.120966
[Epoch] 991  | batch 0 | Loss 2.193309
[Epoch] 992  | batch 0 | Loss 1.825473
[Epoch] 993  | batch 0 | Loss 1.676007
[Epoch] 994  | batch 0 | Loss 1.581259
[Epoch] 995  | batch 0 | Loss 2.015757
[Epoch] 996  | batch 0 | Loss 2.100486
[Epoch] 997  | batch 0 | Loss 2.099599
[Epoch] 998  | batch 0 | Loss 2.374251
[Epoch] 999  | batch 0 | Loss 1.763730


In [11]:
perturb_clf_obj.predict(train_X[-2:])

[array([ 188, 1856,  608,  196,   29,   35,   40,  956]),
 array([ 152, 2281,  301,  167,   22,   81,   24, 1040])]